# Applications of AI in Financial Services

Deep learning is one of the most exciting new technologies being used in the financial services industry, and when used correctly, can improve investment returns. While tasks such as computer vision and natural language processing (NLP) are well-researched areas, the use of Artificial Intelligence (AI) techniques in financial services is still growing. It's important to note that some of the most advanced, lucrative deep learning techniques in AI are not published, nor will they ever be. The lucrative nature of the financial services space necessitates guarding advanced algorithms and measures, and so in this chapter we will focus on principles. 

The application of AI in the financial services industry is nuanced; it's being used in areas where it can perform faster and better than a human could, but still isn't being used ubiquitously. By far, the most ubiquitous use of deep learning in the financial services industry is in feature engineering.

An entire book could be written simply on the topic of deep learning in financial services. While we won't go into depth on financial topics, we will touch upon the definitions of terms and concepts that we introduce throughout this chapter. We'll be covering several basic AI-driven trading methods, as well as an event-based trading method that utilizes a new type of Artificial Neural Network (ANN) that we have yet to talk about—the Neural Tensor Network. Lastly, we'll look at how deep learning can aid us in developing optimal portfolios of stocks. 

## Building a Trading Platform

Adapation from the Personae Platform by @Ceruleanacg

In [ ]:
import math
from time import time
from enum import Enum

#### Create a class to manage the trading position

In [ ]:
class TradingPosition(object):
   ''' Class that manages the trading position of our platform utilizing the Personae platform'''
    
    def __init__(self, action, buy_price, amount, next_price):
        self.action = action ## Status code for what action our algorithm is taking
        self.amount = amount ## The amount of the trade
        self.buy_price = buy_price ## The purchase price of a trade
        self.current_price = buy_price ## Buy price of the trade
        self.current_value = self.current_price * self.amount
        self.pro_value = next_price * self.amount 
        
    def TradeStatus(self, current_price, next_price, amount):
        ''' Manages the status of a trade that is in action '''
        self.current_price = current_price ## updates the current price variable that is maintained within the class
        self.current_value = self.current_price * amount
        pro_value = next_price * amount

    def BuyStock(self, buy_price, amount, next_price):
        ''' Function to buy a stock '''
        self.buy_price = ((self.amount * self.buy_price) + (amount * buy_price)) / (self.amount + amount)
        self.amount += amount
        self.TradeStatus(buy_price, next_price)

    def SellStock(self, sell_price, amount, next_price):
        ''' Function to sell a stock '''
         self.current_price = sell_price
         self.amount -= amount
         self.TradeStatus(sell_price, next_price)
            
    def HoldStock(self, current_price, next_price):
        ''' Function to hold a stock '''
        self.TradeStatus(current_price, next_price)

#### Create an artificial trader

In [ ]:
class Trader(object):
 ''' An Artificial Trading Agent. From the Personae Platform'''

    def __init__(self, market, cash=100000000.0):
         ## Initialize all the variables we need for our trader
         self.cash = cash ## Our Cash Variable
         self.market = market ## current market values
         self.codes = market.codes ## buy, sell, or hold
         self.positions = [] ## current position held
         self.action_times = 0
         self.initial_cash = cash ## the initial cash value in the account
         self.max_cash = cash * 3
         self.total_rewards = 0
         self.cur_action_code = None
         self.cur_action_status = None
         self.episode_time = 0
         self.history_profits = []
         self.history_baselines = []
         self.action_dic = {ActionCode.Buy: self.buy, ActionCode.Hold: self.hold, ActionCode.Sell: self.sell}
        
    ## CountCode will keep track of how many action codes to handle
    @property
    def CountCodes(self):
        return len(self.codes)

    @property
    def ActionSpace(self):
        return self.CountCodes * 3
 
    ## Define what our total profits are
    @property
    def TotalProfits(self):
        return self.cash + self.holdings_value - self.initial_cash

    ## Define the total value of all currently held assets
    @property
    def HoldingsValue(self):
        holdings_value = 0
        for position in self.positions:
            ## Define holdings value as the total value of all current positions
            holdings_value += position.current_value
            return holdings_value

    def BuyAction(self, action, stock, amount, stock_next):
        ## Check if there is enough cash in the account
        amount = amount if self.cash > stock.close * amount else int(math.floor(self.cash / stock.close))
        if amount > 0:
            ## Check if we already hold a certain security
            if not self._exist_position(action):
                ## If we do not own the security, feed the information needed to purchase it to our TradingPosition Class
                position = TradingPosition(action, stock.close, amount, stock_next.close)
                self.positions.append(position) ## Add the new security to our list of securities owned
            else:
                # Get position and update if possible.
                position = self._position(action)
                position.BuyStock(stock.close, amount, stock_next.close)
                
            ## Udate our current cash on hand
            self.cash -= amount * stock.close
            self._update_reward(ActionCode.Buy, ActionStatus.Success, position)
            
            else:
                 self.market.logger.info("Code: {}, insufficient cash reserves.".format(code))
        if self._exist_position(code):
            position = self._position(code)
            position.update_status(stock.close, stock_next.close)
            self._update_reward(ActionCode.Buy, ActionStatus.Failed, position)
        
    def SellAction(self, code, stock, amount, stock_next):
            ## First, check to see if we own the secutity in questions, if not, return an error
            if not self._exist_position(code):
                self.market.logger.info("Code: {}, does not exits in your account".format(code))
                return self._update_reward(ActionCode.Sell, ActionStatus.Failed, None)           
            
            ## Otherwise, attempt to sell the stock 
            position = self._position(code)
            amount = amount if amount < position.amount else position.amount
            position.sub(stock.close, amount, stock_next.close)
 
            ## Lastly, update the amount of cash we now have on hand
            self.cash += amount * stock.close
            self._update_reward(ActionCode.Sell, ActionStatus.Success, position)
    
        def HoldAction(self, code, stock, _, stock_next):
            if not self._exist_position(code):
                self.market.logger.info("Code: {}, you do not own this stock".format(code))
                return self._update_reward(ActionCode.Hold,         ActionStatus.Failed, None)
     
            position = self._position(code)
            position.update_status(stock.close, stock_next.close)
            self._update_reward(ActionCode.Hold, ActionStatus.Success, position)

#### Class for handling the trader's interaction with market data

In [ ]:
class MarketHandler(object):
    ''' Class for handling our platform's interaction with market data. From the Personae Platform'''
    Running = 0
    Done = -1

    def __init__(self, codes, start_date="2008-01-01", end_date="2018-05-31", **options):
         self.codes = codes
         self.index_codes = []
         self.state_codes = []
         self.dates = []
         self.t_dates = []
         self.e_dates = []
         self.origin_frames = dict()
         self.scaled_frames = dict()
         self.data_x = None
         self.data_y = None
         self.seq_data_x = None
         self.seq_data_y = None
         self.next_date = None
         self.iter_dates = None
         self.current_date = None

         ## Initialize the stock data that will be fed in 
         self._init_data(start_date, end_date)

         self.state_codes = self.codes + self.index_codes
         self.scaler = [scaler() for _ in self.state_codes]
         self.trader = Trader(self, cash=self.init_cash)
         self.doc_class = Stock if self.m_type == 'stock' else Future
            
    def _init_data_frames(self, start_date, end_date):
        self._validate_codes()
        columns, dates_set = ['open', 'high', 'low', 'close', 'volume'], set()
        ## Load the actual data
        for index, code in enumerate(self.state_codes):
            instrument_docs = self.doc_class.get_k_data(code, start_date, end_date)
            instrument_dicts = [instrument.to_dic() for instrument in instrument_docs]
            dates = [instrument[1] for instrument in instrument_dicts]
            instruments = [instrument[2:] for instrument in instrument_dicts]
            dates_set = dates_set.union(dates)
            scaler = self.scaler[index]
            scaler.fit(instruments)
            instruments_scaled = scaler.transform(instruments)
            origin_frame = pd.DataFrame(data=instruments, index=dates, columns=columns)
            scaled_frame = pd.DataFrame(data=instruments_scaled, index=dates, columns=columns)
            self.origin_frames[code] = origin_frame
            self.scaled_frames[code] = scaled_frame
            self.dates = sorted(list(dates_set))
        for code in self.state_codes:
            origin_frame = self.origin_frames[code]
            scaled_frame = self.scaled_frames[code]
            self.origin_frames[code] = origin_frame.reindex(self.dates, method='bfill')
            self.scaled_frames[code] = scaled_frame.reindex(self.dates, method='bfill')
        
    def _init_env_data(self):
        if not self.use_sequence:
            self._init_series_data()
        else:
            self._init_sequence_data()
            self._init_data_frames(start_date, end_date)

## Price Prediction Utilizing LSTMs

Adapation from the Personae Platform by @Ceruleanacg

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import logging
import os

In [ ]:
class TradingRNN():
     ''' An RNN Model for Derivatives Training '''
    def __init__(self, session, env, seq_length, x_space, y_space, **options):
        self.seq_length, self.x_space, self.y_space = seq_length, x_space, y_space

    try:
        self.hidden_size = options['hidden_size']
    except KeyError:
         self.hidden_size = 1
            
    self.x = tf.placeholder(tf.float32, [None, self.seq_length, self.x_space])
    self.label = tf.placeholder(tf.float32, [None, self.y_space])
        
    with tf.variable_scope('network_body'):
        self.rnn = self.add_rnn(1, self.hidden_size)
        self.rnn_output, _ = tf.nn.dynamic_rnn(self.rnn, self.x, dtype=tf.float32)
        self.rnn_output = self.rnn_output[:, -1]
        self.rnn_output_dense = self.add_fc(self.rnn_output, 16)
        self.y = self.add_fc(self.rnn_output_dense, self.y_space)
    
    
    with tf.variable_scope('loss'):
        self.loss = tf.losses.mean_squared_error(self.y, self.label)
    with tf.variable_scope('train'):
        self.global_step = tf.Variable(0, trainable=False)
        self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate)
        self.train_op = self.optimizer.minimize(self.loss)
        self.session.run(tf.global_variables_initializer())
        
        
    def train(self):
        for step in range(self.train_steps):
            batch_x, batch_y = self.env.get_batch_data(self.batch_size)
            _, loss = self.session.run([self.train_op, self.loss],             feed_dict={self.x: batch_x, self.label: batch_y})
        if (step + 1) % 1000 == 0:
            logging.warning("Step: {0} | Loss: {1:.7f}".format(step + 1, loss))
        if step > 0 and (step + 1) % self.save_step == 0:
             if self.enable_saver:
                 self.save(step)
            
    def predict(self, x):
         return self.session.run(self.y, feed_dict={self.x: x})

    def main(args):
        mode = 'test'
        codes = ["600036", "601998"]
        market = args.market
        train_steps = 20000
        training_data_ratio = 0.98

        env = Market(codes, start_date="2008-01-01", end_date="2018-01-01", **{
            "market": market,
            "use_sequence": True,
            "scaler": MinMaxScaler,
            "mix_index_state": True,
            "training_data_ratio": training_data_ratio,
        })

        model_name = os.path.basename(__file__).split('.')[0]

        RNN = TradingRNN(tf.Session(config=config), env, env.seq_length, env.data_dim, env.code_count, **{
            "mode": mode,
            "hidden_size": 5,
            "enable_saver": True,
            "train_steps": train_steps,
            "enable_summary_writer": True,
            "save_path": os.path.join(CHECKPOINTS_DIR, "SL",         model_name, market, "model"),
            "summary_path": os.path.join(CHECKPOINTS_DIR, "SL", model_name, market, "summary"),
         })

        RNN.run()
        RNN.eval_and_plot()
    
    if __name__ == '__main__':
         main(model_launcher_parser.parse_args())

## Deep Learning in Asset Management

The autoencoder in this example is adapted from that by time series autoencoder by @/RobRomijnders

In [ ]:
import numpy as np
import tensorflow as tf from tensorflow.contrib.rnn import LSTMCell

#### Parse the Index Data

In [ ]:
ibb = defaultdict(defaultdict)
ibb_full = pd.read_csv('data/ibb.csv', index_col=0).astype('float32')

ibb_lp = ibb_full.iloc[:,0] 
ibb['calibrate']['lp'] = ibb_lp[0:104]
ibb['validate']['lp'] = ibb_lp[104:]

ibb_net = ibb_full.iloc[:,1] 
ibb['calibrate']['net'] = ibb_net[0:104]
ibb['validate']['net'] = ibb_net[104:]

ibb_percentage = ibb_full.iloc[:,2] 
ibb['calibrate']['percentage'] = ibb_percentage[0:104]
ibb['validate']['percentage'] = ibb_percentage[104:]

#### Develop an Autoencoder to Encode the Market Data
Adapted from @RobRomijnders

In [ ]:
class MarketEncoder():
    ''' AutoEncoder for Data Drive Portfolio Allocation '''
    def __init__(self):
        ## Hyperparameters frot the market encoder
        self.layers = tf.placeholder('int') ## Number of layers
        self.hl = tf.placeholder('int') ## Size of the hidden layers
        self.maximum_gradient = tf.placeholder('int') ## The Maximum Gradient
        self.batch = tf.placeholder('int')
        self.crd = tf.placeholder('int') 
        self.num_l = tf.placeholder('int')
        self.lr = tf.placeholder('float') ## Learning Rate
        self.batch_size = batch_size 
        
        ## sl will represent the length of an input sequence, which we would like to eb dynamic based on the data 
        sl = tf.placeholder("int")
        self.sl = sl
        
        ## X will be a placeholder to represent our input data
        self.x = tf.placeholder("float", shape=[batch_size, self.sl], name='input')
        self.x_exp = tf.expand_dims(self.x, 1)
        self.keep_prob = tf.placeholder("float")
        
    ## Layer to Generate the Initial Hidden State from the Encoder
    with tf.name_scope("Initial_State") as scope:
        ## Weights Parameter State
        weights_state = tf.get_variable([num_l, hidden_size])

        ## Bias Paramter State
        bias_state = tf.get_variable('b_state', [hidden_size])
 
        ## Hidden State
        hidden_state= tf.nn.xw_plus_b(self.z_mu, W_state, b_state, name='hidden_state')

    ## Create the Encoder 
    def encoder(self):
        '''Encoder Operation for the autoencoder'''
        
        ## For the encoder, we will use an LSTM cell with Dropout
        EncoderCell = tf.contrib.rnn.MultiRNNCell([LSTMCell(self.hl) for _ in range(layers)])
        EncoderCell = tf.contrib.rnn.DropoutWrapper(EncoderCell, output_keep_prob=self.keep_prob)

        ## Set the initial hidden state of the encoder
        EncInitialState = EncoderCell.zero_state(batch_size, tf.float32)

        ## Weights Factor
        weights = tf.get_variable('weights', [self.hl, num_l])

        ## Outputs of the Encoder Layer
        outputs_enc, _ = tf.contrib.rnn.static_rnn(cell_enc, inputs=tf.unstack(self.x_exp, axis=2),
        initial_state=initial_state_enc)
        cell_output = outputs_enc[-1]

        ## Bias Factor
        biases = tf.get_variable('biases', [num_l])
 
        ## Mean of the latent space variables
        z = tf.nn.xw_plus_b(cell_output, weights, biases) 

        lat_mean, lat_var = tf.nn.moments(self.z_mu, axes=[1])
        self.loss_lat_batch = tf.reduce_mean(tf.square(lat_mean) + lat_var - tf.log(lat_var) - 1)
        
    ## Decoder Layer 
    def decoder(self):  
        ''' Decoder operations for the AE'''
 
        ## Create a base decoder cell
        DecoderCell = tf.contrib.rnn.MultiRNNCell([LSTMCell(self.hl) for _ in range(self.layers)])

        ## Set an initial state for the decoder layer
        DecState = tuple([(hidden_state, hidden_state)] * self.layers)
        decIn = [tf.zeros([batch_size, 1])] * self.sl
 
        ## Run the decoder layer
        outputs_dec, _ = tf.contrib.rnn.static_rnn(cell_dec, inputs=dec_inputs, initial_state=DecState)     
            
    def output_layer(self):
        '''A dense layer that acts as the output layer of the encoder'''
        
        params_o = 2 * crd 
        W_o = tf.get_variable('W_o', [hidden_size, params_o])
        b_o = tf.get_variable('b_o', [params_o])
        outputs = tf.concat(outputs_dec, axis=0) 
        h_out = tf.nn.xw_plus_b(outputs, W_o, b_o)
        h_mu, h_sigma_log = tf.unstack(tf.reshape(h_out, [sl, batch_size, params_o]), axis=2)
        h_sigma = tf.exp(h_sigma_log)
        dist = tf.contrib.distributions.Normal(h_mu, h_sigma)
        px = dist.log_prob(tf.transpose(self.x))
        loss_seq = -px
        self.loss_seq = tf.reduce_mean(loss_seq)
    
        
    def train(self):
        '''Training Function'''
 
        ## Global Step Function for Training
        global_step = tf.Variable(0, trainable=False)
 
        ## Exponential Decay for the learning rate. Takes the initial LR and decays over time
        lr_delta = tf.train.exponential_decay(self.lr, global_step, 1000, 0.1, staircase=False)

        ## Loss Function for the Network
        self.loss = self.loss_seq + self.loss_lat_batch
 
        ## Utilize gradient clipping to prevent exploding gradients
        grads = tf.gradients(self.loss, tvars)
        grads, _ = tf.clip_by_global_norm(grads, self.maximum_gradient)
        self.numel = tf.constant([[0]])

        ## Lastly, apply the optimization process
        optimizer = tf.train.AdamOptimizer(lr_delta)
        gradients = zip(grads, tvars)
        self.train_step = optimizer.apply_gradients(gradients, global_step=global_step)
        self.numel = tf.constant([[0]])

#### Train the autoencoder on the market data

In [ ]:
start = 0
label = [] # The label to save to visualize the latent space
z_run = []

while start + batch_size < Nval:
    run_ind = range(start, start + batch_size)
    z_mu_fetch = sess.run(model.z_mu, feed_dict={model.x: X_val[run_ind], model.keep_prob: 1.0})
    z_run.append(z_mu_fetch)
    start += batch_size

z_run = np.concatenate(z_run, axis=0)
label = y_val[:start]


saver = tf.train.Saver()
saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"), step)
config = projector.ProjectorConfig()

embedding = config.embeddings.add()
embedding.tensor_name = model.z_mu.name

communal_information = []

for i in range(0,83):
    diff = np.linalg.norm((data.iloc[:,i] - reconstruct[:,i])) # 2 norm difference
    communal_information.append(float(diff))
 
print("stock #, 2-norm, stock name")
ranking = np.array(communal_information).argsort()
for stock_index in ranking:
    print(stock_index, communal_information[stock_index], stock['calibrate']['net'].iloc[:,stock_index].name)

    if True:
        sess.run(model.init_op)
        writer = tf.summary.FileWriter(LOG_DIR, sess.graph) # writer for Tensorboard

        step = 0 # Step is a counter for filling the numpy array perf_collect
    for i in range(max_iterations):
        batch_ind = np.random.choice(N, batch_size, replace=False)
        result = sess.run([model.loss, model.loss_seq, model.loss_lat_batch, model.train_step],
        feed_dict={model.x: X_train[batch_ind], model.keep_prob: dropout})

    if i % plot_every == 0:
        perf_collect[0, step] = loss_train = result[0]
        loss_train_seq, lost_train_lat = result[1], result[2]

    batch_ind_val = np.random.choice(Nval, batch_size, replace=False)

    result = sess.run([model.loss, model.loss_seq, model.loss_lat_batch, model.merged],
    feed_dict={model.x: X_val[batch_ind_val], model.keep_prob: 1.0})
    perf_collect[1, step] = loss_val = result[0]
    loss_val_seq, lost_val_lat = result[1], result[2]
    summary_str = result[3]
    writer.add_summary(summary_str, i)
    writer.flush()

    print("At %6s / %6s train (%5.3f, %5.3f, %5.3f), val (%5.3f, %5.3f,%5.3f) in order (total, seq, lat)" % (
 i, max_iterations, loss_train, loss_train_seq, lost_train_lat, loss_val, loss_val_seq, lost_val_lat))
 step += 1
if False:

#### Reconconstruct the Index Utilizing the Autoencoder

In [ ]:
which_stock = 1

stock_autoencoder = copy.deepcopy(reconstruct[:, which_stock])
stock_autoencoder[0] = 0
stock_autoencoder = stock_autoencoder.cumsum()
stock_autoencoder += (stock['calibrate']['lp'].iloc[0, which_stock])

pd.Series(stock['calibrate']['lp'].iloc[:, which_stock].as_matrix(), index=pd.date_range(start='01/06/2012', periods=104, freq='W')).plot(label='stock original', legend=True)
pd.Series(stock_autoencoder, index=pd.date_range(start='01/06/2012', periods = 104,freq='W')).plot(label='stock autoencoded', legend=True)